In [74]:
! pip install -U openai-whisper
! pip install pandas
! pip install convokit
! pip install nltk
! pip install spacy
! python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
/Users/jamesphoughton/github/deliberation-video-analysis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23

# Local Model

In [72]:
import whisper
import pandas as pd
import glob
import os
import re
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jamesphoughton/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
folder = "testVideos/constructive_disagreement/r20230929_1040_student_011/"
files = glob.glob(folder + "*-audio-*")

for file in files:
    if not file.endswith("webm"):
        os.rename(file, file + ".webm")

files = glob.glob(folder + "*-audio-*")
files

['testVideos/constructive_disagreement/r20230929_1040_student_011/1695991149876-b31d0273-aadd-443a-bf33-2589234552c5-cam-audio-1695991150760.webm',
 'testVideos/constructive_disagreement/r20230929_1040_student_011/1695991149876-8c3fb626-81ac-460c-b782-ae967c43a649-cam-audio-1695991155119.webm',
 'testVideos/constructive_disagreement/r20230929_1040_student_011/1695991149876-2cd84f16-9fe9-4c17-b882-fe40a1b023fb-cam-audio-1695991243321.webm']

In [59]:
def getStart(filepath):

    probe = ! ffprobe $filepath 
    line = [line  for line in probe if "start: " in line][0]
    pattern = r"start:\s(\d+\.\d+)"
    match = re.search(pattern, line)
    if match:
        start = float(match.group(1))
    else:
        console.log("No start time listed, using 0.0")
        start = 0.0
    return start


In [60]:
model = whisper.load_model("base")

def transcribe(filepath):

    start = getStart(filepath)
    result = model.transcribe(filepath)
    df = pd.DataFrame(result['segments'])
    df["start"] = df['start'] + start
    df["end"] = df['end'] + start

    return df[["start", "end", "text"]]



In [67]:
res = [transcribe(filepath) for filepath in files ]

/Users/jamesphoughton/github/deliberation-video-analysis/.venv/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/jamesphoughton/github/deliberation-video-analysis/.venv/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/jamesphoughton/github/deliberation-video-analysis/.venv/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [68]:
for i, df in enumerate(res):
    df.set_index("start", inplace=True)
    df['speaker'] = i

transcript = pd.concat(res).sort_index()


In [69]:
transcript

,end,text,player
start,,,
3.360,8.360,"Hello, finally.",0
5.201,12.201,"Hello, director of ethics.",1
8.360,11.360,See you.,0
11.360,15.360,Congratulations on your promotion.,0
12.201,17.201,"Oh, you can hear me.",1
...,...,...,...
422.401,425.401,And that's one of the criteria that we are as...,1
425.360,429.360,"Yeah, I mean, it's good that he raised money,...",0
429.360,435.360,I don't see anything regarding additional tra...,0


# Word timings:

In [36]:
model = whisper.load_model("base")
result = model.transcribe(filename, word_timestamps=True)
# pd.DataFrame(result['segments'])
result

/Users/jamesphoughton/github/deliberation-video-analysis/.venv/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': " Hi guys, my name is I think the points that we have been shared are same right? Because I realize some of the other guys have got the additional insights for everyone is different. Could you read your points Dylan could you read me the resume highlights and additional insights just once so that I can match it? Because I realize that all of us have been given different points as such. Okay. Okay. Yes. Yes. Okay. Okay. Now, resume highlights is same for everyone. I guess what is the. Okay. The first one is national reputation and the last one has been observed drinking. Is it the same? Yeah. So, and us. Yes. Yeah. So, I'm at could you please read out the additional insights will get to know if any of us have different insights. Okay. Okay. Okay. No. No. Okay. Discarriage new innovative ideas. Okay. And I have got a few extra engages in IT consulting work exhibited a strong or at least skills during faculty presentation likes to garden. Okay. Okay. Okay. So, I think nothing rea